In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import torch
import pyro
from tqdm.notebook import tqdm
print(f"Pyro version: {pyro.__version__}")

# An introduction to Gaussian processes

## Motivation

Before training a neural network (deterministic or probabilistic) we have to **select its architecture**. 

For example, in the case of a multilayer perceptron (MLP), we need to choose the number of layers (depth) and the number of nodes (neurons) per layer (width)

> With this we are defining the neural network as a **function with fixed structure**

Increasing the width and depth gives the model more flexibility. But in general we don't know "how much" flexibility is needed for a particular problem

The architecture is a collection of hyper-parameters. The good practice is to find the "best structure" using a validation (holdout) dataset

Instead of testing several architectures we could use a non-parametric model with no fixed structure

In this lesson we will study the **Gaussian Process (GP)**, a bayesian non-parametric model that can be seen as a neural network with one hidden layer and potentially infinite width

> In general non-parametric models automatically grow in complexity (width) with data

Note that non-parametric models do have prior distributions and tunable hyper-parameters. The difference is that the distribution of its parameters lives in an infinite dimensional space. We use non-parametric models by integrating out (marginalizing) this infinite distribution

Other non-parametric models not covered in this lesson are the many variants of the Dirichlet Process, the infinite Hidden Markov Model and the Indian Buffer Process. See Ghahramani's tutorial at the end of this lesson for a presentation of these methods

## Theorical background 

### What is a Gaussian process?

Consider the probabilistic linear regression problem from previous lessons

$$
y_i =  \sum_{k=1}^M \phi_k(x_i) \theta_k + \epsilon_i \quad \forall i=1,2,\ldots,N
$$

where $\epsilon_i$ is *iid* Gausian and $\phi_k$ is a collection of $M$ (non-linear) basis functions

We can write this in matrix form as

$$
Y = \Phi(X) \theta + E
$$

where $E$ is a diagonal matrix and $\Phi(X) \in \mathbb{R}^{N \times M}$

The prior for $\theta$ is Gaussian

$$
p(\theta) = \mathcal{N}(0,  \sigma_\theta^2 I)
$$

We may ask 

> What is the distribution of $f_\theta(X) = \Phi(X) \theta$ ?

If $\Phi$ is a deterministic transformation then the distribution of $f$ is also Gaussian 

By our previous definitions the mean of $p(f)$ is 

$$
\mathbb{E}[f_\theta(X)] = \Phi(X)\mathbb{E}[\theta] = 0
$$

and its covariance is

$$
\mathbb{E}[ f_\theta(X) f_\theta(X)^T] =  \Phi(X)\mathbb{E}[\theta \theta^T ] \Phi(X)^T = \sigma_\theta^2 \Phi(X) \Phi(X)^T = K
$$

where $K \in \mathbb{R}^{N\times N}$ is a symmetric and positive-definite matrix called the **Gram matrix** or **Gramian matrix**.

The $ij$-th element of the gram matrix is

$$
K_{ij} = \sum_{k=1}^M \phi_k(x_i) \phi_k(x_j) = \left \langle \vec \phi(x_i) , \vec \phi(x_j) \right \rangle = \kappa(x_i, x_j)
$$

where $\kappa(\cdot, \cdot): \mathcal{X} \times \mathcal{X} \to \mathbb{R}$ is called the **kernel**. In general we will forget about $\{\phi_k(\cdot)\}$ and work only with the kernel (more about this later).

With all this we can finally write

$$
p(f) = \mathcal{N}(0, K)
$$

which is a "prior over functions". Note that we have dropped the dependence on $\theta$

We can say that $f$ is a multivariate random variable or "random process" with joint Gaussian distribution: $f$ is a **Gaussian Process**

### How do we do inference with a GP?

Let's say we have a dataset $\textbf{x}=(x_1, x_2)$, $\textbf{y}=(y_1, y_2)$ and we want to infer $f(x^*)$ or $f^*$ for short, i.e. we are interested in the posterior $p(f^*|\textbf{y}, \textbf{x}, x^*)$

As before we can write the joint (Gaussian) distribution between the dataset and the new sample as

$$
p(\textbf{y}, f^*) = \mathcal{N}(0, K^+)
$$

where

$$
K^+ = \begin{pmatrix} K_{\textbf{x}\textbf{x}} + \sigma_\epsilon^2 I & K_{\textbf{x}x^*} \\ K_{\textbf{x}x^*}^T & K_{x^*x^*} \end{pmatrix}
$$

is a block matrix and

$$
K_{\textbf{x}\textbf{x}} = \begin{pmatrix} \kappa(x_1, x_1) & \kappa(x_1, x_2) \\ \kappa(x_1, x_2) & \kappa(x_2, x_2)\end{pmatrix}, \quad K_{\textbf{x}x^*} = \begin{pmatrix} \kappa(x_1, x^*) \\ \kappa(x_2, x^*) \end{pmatrix}
$$

The Gaussian distribution is closed under conditioning, i.e. if we have a joint gaussian distribution the conditional distribution of a variable given the others is gaussian ([nice step by step example](https://fabiandablander.com/statistics/Two-Properties.html))

Here we will use this property to write 

$$
\begin{align}
p(f^*|\textbf{y}, \textbf{x}, x^*) = \mathcal{N}(&K_{\textbf{x}x^*} (K_{\textbf{x}\textbf{x}}+I\sigma_\epsilon^2)^{-1} \textbf{y}, \nonumber \\
& K_{x^*x^*} - K_{\textbf{x}x^*} (K_{\textbf{x}\textbf{x}}+I\sigma_\epsilon^2)^{-1} K_{\textbf{x}x^*}^T ) 
\end{align}
$$

which gives us the result we seek

We can use Gaussian conditioning to predict on several "new data points" at the same time, we only need to compute the sub gram matrices between and within the training set and the test set

<img src="images/gram_matrix_block.png" width="300">

### More about the kernel

The GP is mainly defined by its covariance also known as the gram matrix

$$
K = \begin{pmatrix} 
\kappa(x_1, x_1)& \kappa(x_1, x_2)& \ldots & \kappa(x_1, x_N) \\
\kappa(x_2, x_1)& \kappa(x_2, x_2)& \ldots & \kappa(x_2, x_N) \\
\vdots& \vdots& \ddots & \vdots \\
\kappa(x_N, x_1)& \kappa(x_N, x_2)& \ldots & \kappa(x_N, x_N) \\
\end{pmatrix}
$$

where the following relation between the kernel and the basis function

$$
\kappa(x_i, x_j) = \left \langle \vec \phi(x_i) , \vec \phi(x_j) \right \rangle 
$$

is known as the "[kernel trick](https://en.wikipedia.org/wiki/Kernel_method#Mathematics:_the_kernel_trick)". 

Before we defined a finite dimensional $\vec \phi$ and obtained $\kappa$. But in general it is more interesting to skip $\phi$ and design $\kappa$ directly. We only need $\kappa$ to be a symmetric and positive-definite function. The broadly used Gaussian kernel complies with these restrictions

$$
\kappa(x_i, x_j) = \sigma^2 \exp \left ( \frac{\|x_i - x_j \|^2}{2\ell^2} \right)
$$

where hyperparameter $\sigma$ controls the amplitude and $\ell$ controls the length-scale of the interactions between samples. 

Using a taylor expansion we can show that the (non-linear) basis function of this kernel is 

$$
\phi(x) = \lim_{M\to\infty} \sigma^2 \exp\left({-\frac{\|x\|^2}{2\ell^2}}\right) \begin{pmatrix} 1 & \frac{x}{\ell} & \frac{x^2}{\ell^2 \sqrt{2}} & \cdots & \frac{x^M}{\ell^M \sqrt{M!}}  \end{pmatrix}
$$

i.e. the Gaussian kernel induces an infinite-dimensional basis function. 

> A Gaussian process with Gaussian kernel has an implicit infinite dimensional parameter vector $\theta$. 

We are not anymore explicitely choosing the structure of the function, but we selecting a kernel we are choosing a general "behavior". For example the Gaussian kernels encodes the property of locality, i.e. closer data samples should have similar predictions. 

Several other valid kernels exist that encode other properties such as trends and periodicity as exemplified in the following picture from Mackay's book

<img src="images/kernels_mackay.png" width="800">

### How do we train a GP?

Fitting a GP to a dataset corresponds to finding the best combination of kernel hyperparameters

We can do this by maximizing the marginal likelihood. For regression with iid Gaussian noise the marginal likelihood $y$ is also Gaussian

$$
p(\textbf{y}|\textbf{x}) = \int p(\textbf{y}|f) p(f) \,df = \mathcal{N}(0, K + \sigma_\epsilon^2 I )
$$

where the hyperparamter $\sigma_\epsilon^2$ is the variance of the noise 

It is equivalent and much easier to maximize the log marginal likelihood

$$
\log p(\textbf{y}|\textbf{x}) = -\frac{1}{2} \textbf{y}^T (K + \sigma_\epsilon^2I)^{-1} \textbf{y} - \frac{1}{2} \log | 2\pi (K + \sigma_\epsilon^2I) |
$$

from which we compute derivatives to update the hyperparameters through gradient descent

The following picture from Barber's book shows three examples drawn from the GP prior (gaussian kernel) on the left and the mean/variance of the GP posterior after training on the right

<img src="images/gp_fitted.png" width="700">

## Gaussian Processes with `pyro`

### Setting, training and performing inference 

Let's start by creating some synthetic data for regression

In [ ]:
# Synthetic data
se = 0.1
np.random.seed(0)
x = np.linspace(0, 1, num=20) #100x1
x_test = np.linspace(-0.4, 1.6, num=200).astype('float32')
f = lambda x : x*np.sin(10*x)

x = np.delete(x, slice(9, 14))
y = f(x) + se*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.scatter(x, y);

x_torch = torch.from_numpy(x.astype('float32'))
y_torch = torch.from_numpy(y.astype('float32'))

We will use [`pyro.contrib.gp`](http://docs.pyro.ai/en/stable/contrib.gp.html) to implement our first GP

Let's start by creating a kernel from `gp.kernels`

We will use a Radial Basis Function (RBF) aka Squared Exponential aka Gaussian kernel as our covariance

We can specify the initial value of the variance and the lengthscale

In [ ]:
import pyro.contrib.gp as gp

pyro.enable_validation(True)
pyro.set_rng_seed(0)

K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.), 
                   lengthscale=torch.tensor(0.1))

How does this model looks before fitting the data? 

Let's inspect the prior $p(f) = \mathcal{N}(0, K)$ on the test data

**Activity:** Increase/decrese the lengthscale and repeat, get a notion of its influence

In [ ]:
# We sum a small value to the diagonal for numerical stability
C = K.forward(torch.from_numpy(x_test)) + torch.eye(len(x_test))*1e-3
# Then we sample from the a multivariate normal distribution
samples = pyro.distributions.MultivariateNormal(torch.zeros(len(x_test)), 
                                                covariance_matrix=C).sample(sample_shape=(50,))
        
fig, ax = plt.subplots(figsize=(6, 3))
for i in range(samples.shape[0]):
    ax.plot(x_test, samples.detach().numpy()[i, :],
            linestyle='-', c='tab:blue', alpha=0.5)

In [ ]:
# Helper function to train GP and plot the results

def train_gp_plots(model, x, y, x_test, nepochs=2000):
    fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
    line_loss = ax[1].plot([], [])
    ax[0].scatter(x, y)
    epoch_loss = np.zeros(shape=(nepochs,))

    for k in tqdm(range(len(epoch_loss))):
        optimizer.zero_grad()
        loss = criterion(model.model, model.guide)
        loss.backward()
        optimizer.step()
        epoch_loss[k] = loss.item()
        #break    
        if k % 100 == 0:
            ax[0].cla()
            # Predictions at x_test
            mu, cov = model.forward(x_test, full_cov=True, noiseless=False)
            mu = mu.detach().numpy()
            sd = cov.diag().sqrt().detach().numpy()        
            ax[0].scatter(x, y, c='k')
            ax[0].plot(x_test.detach(), mu)
            ax[0].fill_between(x_test.detach(), mu-2*sd, mu+2*sd, alpha=0.5)
            line_loss[0].set_xdata(range(k))
            line_loss[0].set_ydata(epoch_loss[:k])
            ax[1].relim()
            ax[1].autoscale_view()
            fig.canvas.draw()

Then we create a model from `gp.models`. For regression there is `GPRegression` and for classification (non-gaussian likelihoods) we can use `VariationalGP`. There are also more efficient (sparse) versions of both models

The model expects 

- the train data
- the kernel 
- initial value of the noise variance

We may also specify a mean function for the GP

To train the model we have to select an optimizer and a cost function. We will use Adam and the Trace_ELBO, respectively

Training is very similar to how we train neural networks in pytorch

In [ ]:
pyro.clear_param_store()

#Kernel
K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.0), 
                   lengthscale=torch.tensor(0.1))
# Model
gpr_model = gp.models.GPRegression(x_torch, y_torch, # Training data
                                   mean_function=None, # Mean
                                   kernel=K, # Covarianze
                                   jitter=1e-6, # Increase this if you have numerical problems 
                                   noise=torch.tensor(2.) # The variance of the white noise
                                   )
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss

train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

The learned parameters are

In [ ]:
display("RBF variance:", gpr_model.kernel.variance.item())
display("RBF length scale:", gpr_model.kernel.lengthscale.item())
display("Noise variance:", gpr_model.noise.item())

To do predictions we use the forward attribute of our `GPRegression` instance

In [ ]:
# We sum a small value to the diagonal for numerical stability
mu, Sigma = gpr_model.forward(torch.from_numpy(x_test), full_cov=True, noiseless=True)
Sigma += torch.eye(len(x_test))*1e-5
# Then we sample from the a multivariate normal distribution
samples = pyro.distributions.MultivariateNormal(mu, covariance_matrix=Sigma).sample(sample_shape=(50,))
        
fig, ax = plt.subplots(figsize=(6, 3))
for i in range(samples.shape[0]):
    ax.plot(x_test, samples.detach().numpy()[i, :], 
            linestyle='-', c='tab:blue', alpha=0.25)
ax.scatter(x, y, c='k', zorder=100);

### Trying different kernels

Kernel are implemented in [pyro.contrib.gp.kernels](http://docs.pyro.ai/en/stable/contrib.gp.html#module-pyro.contrib.gp.kernels)

Compare the RBF and Matern52 kernels. What differences do you observe?

In [ ]:
pyro.clear_param_store()

#Kernel
K = gp.kernels.RBF(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
#K = gp.kernels.Matern52(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
#K = gp.kernels.Periodic(input_dim=1, variance=torch.tensor(2.0), lengthscale=torch.tensor(0.1), period=torch.tensor(0.5))
# Model
gpr_model = gp.models.GPRegression(x_torch, y_torch, 
                                   kernel=K, 
                                   noise=torch.tensor(2.))
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss
# Train and plot
train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

### Setting priors for the parameters 

Before we did an MLE-like estimation to find point estimates of the kernel hyper-parameters and the noise variance

We can "go bayesian" and treat these parameters as random variables and set priors for them

Training with these priors is equivalent to the MAP solution

In [ ]:
pyro.clear_param_store()
from pyro.distributions import LogNormal

#Kernel
K = gp.kernels.RBF(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
# Model
gpr_model_prior = gp.models.GPRegression(x_torch, y_torch, 
                                   kernel=K, 
                                   noise=torch.tensor(2.))

# Setting priors
gpr_model_prior.kernel.lengthscale = pyro.nn.PyroSample(LogNormal(0.0, 1.0))
gpr_model_prior.kernel.variance = pyro.nn.PyroSample(LogNormal(0.0, 1.0))

# Optimizer
optimizer = torch.optim.Adam(gpr_model_prior.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss
# Train and plot    
train_gp_plots(gpr_model_prior, x, y, torch.from_numpy(x_test))

### Combining kernels

Adding or multiplying valid kernels yield a valid kernel

> We can easily create new kernels from other kernels

and take advantage of their different properties

The following data has a periodic oscilation on a rising trend:

In [ ]:
x = np.random.rand(100).astype('float32')*100
y = (0.03*x + np.sin(0.1*x) + 0.1*np.random.randn(100)).astype('float32')

Try to fit it using `K1`, `K2`, `Ksum12` and `Kprod12`

Can you explain in simple words your results?

In [ ]:
pyro.clear_param_store()

K1 = gp.kernels.Periodic(input_dim=1, variance=torch.tensor(1.), 
                        lengthscale=torch.tensor(10),
                        period=torch.tensor(50))
K2 = gp.kernels.Linear(input_dim=1, variance=torch.tensor(1.))
Ksum12 = gp.kernels.Sum(K1, K2)
Kprod12 = gp.kernels.Product(K1, K2)

# Model
gpr_model = gp.models.GPRegression(torch.from_numpy(x), torch.from_numpy(y), 
                                   kernel=K1, noise=torch.tensor(2.))

optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
criterion = pyro.infer.Trace_ELBO().differentiable_loss
train_gp_plots(gpr_model, x, y, torch.linspace(-50, 150, 100), nepochs=1000)

### Sparse Gaussian Processes with `pyro`

Fitting a Gaussian process has cubic complexity

Sparse Gaussian processes use a much smaller set of "inducing points" to compute the kernel

In [ ]:
# Synthetic data
se = 0.1
np.random.seed(0)
x = np.linspace(0, 1, num=1000) #100x1
x_test = np.linspace(-0.1, 1.1, num=200).astype('float32')
f = lambda x : x*np.sin(10*x)

x = np.delete(x, slice(9, 14))
y = f(x) + se*np.random.randn(len(x))

x_torch = torch.from_numpy(x.astype('float32'))
y_torch = torch.from_numpy(y.astype('float32'))

In [ ]:
pyro.clear_param_store()

#Kernel
K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.0), 
                   lengthscale=torch.tensor(0.1))
# Model
#gpr_model = gp.models.GPRegression(x_torch, y_torch, 
#                                   kernel=K, 
#                                   noise=torch.tensor(2.))
gpr_model = gp.models.SparseGPRegression(x_torch, y_torch, approx='VFE',
                                         kernel=K, Xu=torch.linspace(0, 1, 20),
                                         noise=torch.tensor(2.), jitter=1e-5)
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss

train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

## Topics for the future

- Example of GP for classification (Bernoulli likelihood)
- Deep Gaussian Processes/Stacks of Gaussian Processes. Example in Pyro: https://fehiepsi.github.io/blog/deep-gaussian-process/
- [Compositional kernel learning](https://arxiv.org/pdf/1806.04326.pdf)
- Bayesian optimization with Gaussian processes
- https://github.com/team-approx-bayes/dnn2gp

## Self-study

- Mackay's book, chapter 45 on Gaussian Proceses
- Barber's book, chapter 19 on Gaussian Processes
- [Rasmussen & Willams, "Gaussian Process for Machine Learning"](http://gaussianprocess.org/gpml/?)
- [Zhoubin Ghahramadi tutorial](http://mlg.eng.cam.ac.uk/zoubin/talks/uai05tutorial-b.pdf)